In [1]:
!pip install langchain==0.0.274
!pip install gpt4all==1.0.8
!pip install chromadb==0.4.7
!pip install llama-cpp-python
!pip install urllib3==2.0.4
!pip install PyMuPDF==1.23.1
!pip install python-dotenv==1.0.0
!pip install unstructured==0.10.8
!pip install extract-msg==0.45.0
!pip install tabulate==0.9.0
!pip install tqdm==4.66.1
!pip install sentence_transformers
!pip install jq
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.5/415.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 107.3 MB/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 M

In [2]:
# Standard Library Imports
import os
import glob
import time
from multiprocessing import Pool

# Third-Party Library Imports
from typing import List
from dotenv import load_dotenv
from tqdm import tqdm

# Langchain Imports
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PyMuPDFLoader,
    TextLoader,
    JSONLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.llms import GPT4All, LlamaCpp
from transformers import AutoModel, AutoTokenizer,  BertModel, BertTokenizer

# ChromaDB Imports
from chromadb.config import Settings
import chromadb

# Argument Parsing
import argparse

import torch  # Import PyTorch to check GPU availability
from google.colab import drive

In [22]:
persist_directory = "./db/"
#model_type = "GPT4All_GROOVY"
#model_type = "GPT4All_SNOOZY"
#model_type = "VLLM"
model_type = "LLaMA_2_7B"
source_directory = "/content/drive/MyDrive/Colab/SOR/"
embeddings_model_name = "all-MiniLM-L6-v2"
model_n_ctx = 1000
model_n_batch = 8
target_source_chunks = 4
chunk_size = 500
chunk_overlap = 50

In [23]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# List files in the specified source directory
archivos = os.listdir(source_directory)

# Count the number of files in the directory
cantidad_de_archivos = len(archivos)

# Print the number of files in the directory
print(f"The folder '{source_directory}' contains {cantidad_de_archivos} files.")


The folder '/content/drive/MyDrive/Colab/SOR/' contains 2 files.


In [25]:
# Check if GPU is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


In [26]:
# Create embeddings using Hugging Face model
# The 'embeddings_model_name' specifies the pre-trained model to use for embeddings.

# Load the model and tokenizer from transformers, specifying the device
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
# Map file extensions to document loaders and their arguments
LOADER_MAPPING = {
    ".csv": (CSVLoader, {})  # Use the CSVLoader for .csv files with no additional arguments.
    # Add more mappings for other file extensions and loaders as needed
}


In [28]:
# Function to load a single document based on its file path
def load_single_document(file_path: str) -> List[Document]:
    # Extract the file extension from the given file path.
    ext = "." + file_path.rsplit(".", 1)[-1].lower()

    # Check if the file extension is in the LOADER_MAPPING dictionary.
    if ext in LOADER_MAPPING:
        # Get the loader class and loader arguments for the specified extension.
        loader_class, loader_args = LOADER_MAPPING[ext]

        # Create an instance of the loader class with the specified file path and arguments.
        loader = loader_class(file_path, **loader_args)

        # Load the document using the loader and return it.
        return loader.load()

    # If the file extension is not supported, raise a ValueError.
    raise ValueError(f"Unsupported file extension '{ext}'")

In [29]:
# Function to load documents from the source directory
def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]:
    # Find all files in the source directory with extensions specified in LOADER_MAPPING.
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext.lower()}"), recursive=True)
        )
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext.upper()}"), recursive=True)
        )

    # Filter out files that are in the ignored_files list.
    filtered_files = [file_path for file_path in all_files if file_path not in ignored_files]

    # Use a multiprocessing Pool to load documents in parallel.
    with Pool(processes=os.cpu_count()) as pool:
        results = []
        # Create a progress bar for loading documents.
        with tqdm(total=len(filtered_files), desc='Loading new documents', ncols=80) as pbar:
            for i, docs in enumerate(pool.imap_unordered(load_single_document, filtered_files)):
                results.extend(docs)
                pbar.update()

    return results



In [30]:
# Function to process documents into text chunks
def process_documents(ignored_files: List[str] = []) -> List[Document]:
    # Print a message indicating that documents are being loaded from the specified source directory.
    print(f"Loading documents from {source_directory}")

    # Load documents from the source directory, excluding any ignored files.
    documents = load_documents(source_directory, ignored_files)

    # Check if there are no documents to process and exit if that's the case.
    if not documents:
        print("No new documents to load")
        exit(0)

    # Print the number of loaded documents and the source directory.
    print(f"Loaded {len(documents)} new documents from {source_directory}")

    # Create a text splitter with the specified chunk size and overlap.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # Split the loaded documents into chunks of text using the text splitter.
    texts = text_splitter.split_documents(documents)

    # Print the number of text chunks created and the maximum chunk size.
    print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")

    # Return the resulting text chunks.
    return texts

In [31]:
# Function to check if the vector store already exists
def does_vectorstore_exist(persist_directory: str, embeddings: HuggingFaceEmbeddings) -> bool:
    # Create a Chroma vector store instance with the specified persist directory and embeddings.
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

    # Get the 'documents' data from the vector store. If it's empty, return False; otherwise, return True.
    if not db.get()['documents']:
        return False
    return True

In [32]:
# Check if the vector store already exists in the specified directory with the given embeddings.
if does_vectorstore_exist(persist_directory, embeddings):
    # If the vector store exists, append to it.
    print(f"Appending to existing vector store at {persist_directory}")

    # Create a Chroma vector store instance with the specified directory and embeddings.
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

    # Get the existing collection from the vector store.
    collection = db.get()

    # Extract source file paths from the collection's metadata.
    source_file_paths = [metadata['source'] for metadata in collection['metadatas']]

    # Process the documents based on the extracted source file paths.
    texts = process_documents(source_file_paths)

    # Inform the user about the embeddings creation process.
    print(f"Creating embeddings. May take some minutes...")

       # Check if 'texts' is not empty before adding documents to the vector store
    if texts:
        # Add the processed documents to the vector store.
        db.add_documents(texts)
    else:
        print("No documents to add. Skipping insertion.")
else:
    # If the vector store does not exist, create a new one.
    print("Creating a new vector store")

    # Process documents without specifying ignored files (default behavior).
    texts = process_documents()

    # Inform the user about the embeddings creation process.
    print(f"Creating embeddings. May take some minutes...")

    # Create a new Chroma vector store with the processed documents and embeddings.
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)

# Persist the vector store.
db.persist()

# Clear the db variable to free up resources.
db = None

# Inform the user that the ingestion process is complete.
print(f"Ingestion complete! You can now run privateGPT.py to query your documents")


Appending to existing vector store at ./db/
Loading documents from /content/drive/MyDrive/Colab/SOR/


Loading new documents: 0it [00:00, ?it/s]

No new documents to load
Loaded 0 new documents from /content/drive/MyDrive/Colab/SOR/
Split into 0 chunks of text (max. 500 tokens each)
Creating embeddings. May take some minutes...
No documents to add. Skipping insertion.
Ingestion complete! You can now run privateGPT.py to query your documents


In [33]:
# Create settings for Chroma database configuration
settings = Settings(
    persist_directory=persist_directory,  # Directory for persisting database data
    anonymized_telemetry=False  # Disable anonymized telemetry
)

# Create Hugging Face embeddings model
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

# Create a persistent Chroma database client
chroma_client = chromadb.PersistentClient(
    settings=settings,  # Database settings
    path=persist_directory  # Path to the database directory
)

# Create a Chroma vector store instance
db = Chroma(
    persist_directory=persist_directory,  # Directory for persisting vector store data
    embedding_function=embeddings,  # Embeddings function
    client_settings=settings,  # Database settings
    client=chroma_client  # Chroma client
)

# Create a retriever for document retrieval
retriever = db.as_retriever(
    search_kwargs={"k": target_source_chunks}  # Search settings (e.g., number of search results)
)


# Comparación de Modelos de Lenguaje Natural

1. LLaMA 2 (LlamaCpp)

Descripción: LLaMA 2 es una variante del modelo LLaMA, optimizada para funcionar de manera eficiente en distintas plataformas. Utiliza el backend llama-cpp para realizar inferencias de manera rápida.
Características Técnicas:
Ruta del modelo: Especifica la ubicación del modelo LLaMA 2 en el sistema de archivos.
Tokens Máximos: Define el número máximo de tokens generados en una sola inferencia.
Batch Size: Controla el tamaño del lote para la generación de texto.
Número de Hilos: Determina la cantidad de hilos utilizados para la computación, afectando el rendimiento.
Temperatura: Ajusta la aleatoriedad en la generación de texto, influyendo en la creatividad del output.
Ventajas: Alta eficiencia y rendimiento, especialmente en sistemas con limitados recursos.
Desventajas: Requiere un ajuste cuidadoso de los parámetros para obtener resultados óptimos.
2. GPT4All Groovy

Descripción: GPT4All Groovy es una implementación del modelo GPT-4, optimizada para funcionar con el backend gptj, lo que le permite generar texto de alta calidad con una gran variedad de aplicaciones.
Características Técnicas:
Ruta del modelo: Define la ubicación del modelo GPT4All Groovy.
Tokens Máximos: Número máximo de tokens que puede generar en una sesión.
Batch Size: Tamaño del lote utilizado durante la generación de texto.
Backend: Utiliza gptj como backend, conocido por su balance entre rendimiento y calidad.
Temperatura: Controla la aleatoriedad en la generación de texto.
Ventajas: Buena calidad en la generación de texto y flexibilidad en su uso.
Desventajas: Puede ser más demandante en términos de recursos computacionales.
3. GPT4All Snoozy

Descripción: GPT4All Snoozy es otra implementación del modelo GPT-4, también optimizada para el backend gptj. Está diseñado para ofrecer un balance entre rendimiento y precisión.
Características Técnicas:
Ruta del modelo: Ubicación específica del modelo GPT4All Snoozy.
Tokens Máximos: Máximo número de tokens generados.
Batch Size: Tamaño del lote para la generación de texto.
Backend: Utiliza gptj para un rendimiento eficiente.
Temperatura: Control de la aleatoriedad en la generación.
Ventajas: Ofrece un equilibrio entre precisión y rendimiento.
Desventajas: Similar a Groovy, puede ser intensivo en términos de recursos.
4. VLLM

Descripción: VLLM es un modelo basado en la arquitectura MPT (MosaicML Pre-trained Transformer), optimizado para tareas de generación de texto con gran eficiencia.
Características Técnicas:
Modelo: Utiliza mosaicml/mpt-7b, un modelo robusto para diversas tareas.
Tokens Máximos: Controla el máximo de tokens generados.
Top-k: Parámetro que influye en la selección de las próximas palabras, limitando a las k más probables.
Top-p: Probabilidad acumulada que define el conjunto de próximas palabras seleccionables.
Temperatura: Influencia de la aleatoriedad en la generación de texto.
Ventajas: Alta eficiencia y adecuado para tareas que requieren gran cantidad de texto generado.
Desventajas: Requiere un ajuste detallado de los parámetros para maximizar el rendimiento.
Selección del Mejor Modelo
Para seleccionar el mejor modelo, debemos considerar varios factores:

Calidad de la Generación de Texto:

LLaMA 2: Ideal para sistemas con recursos limitados, pero puede requerir ajustes finos para resultados óptimos.
GPT4All Groovy y Snoozy: Ambos ofrecen alta calidad en la generación de texto con flexibilidad en su uso. La elección entre Groovy y Snoozy puede depender de pequeñas diferencias en rendimiento y precisión.
VLLM: Excelente para tareas que requieren generación de grandes volúmenes de texto con eficiencia.
Rendimiento y Recursos Computacionales:

LLaMA 2: Excelente balance entre rendimiento y uso de recursos.
GPT4All Groovy y Snoozy: Pueden ser más demandantes en términos de recursos, pero ofrecen alta calidad en generación.
VLLM: Alta eficiencia, pero puede necesitar ajustes detallados de parámetros.
Flexibilidad y Personalización:

LLaMA 2 y VLLM: Ofrecen gran flexibilidad en términos de ajuste de parámetros como n_threads, top_k, y top_p.
GPT4All Groovy y Snoozy: Flexibles y ajustables, pero pueden requerir más recursos computacionales.

In [34]:
# Initialize an empty list to store callback handlers (you can add handlers here if needed)
callbacks = []

# Prepare the Language Model (LLM) based on the specified model_type
match model_type:
    case "LLaMA_2_7B":
        # Create an instance of LlamaCpp
        llm = LlamaCpp(
            model_path = "/content/drive/MyDrive/Colab/Models/openorca-platypus2-13b.Q4_0.gguf",  # Path to the LlamaCpp model
            max_tokens=model_n_ctx,  # Maximum number of tokens in generated text
            n_batch=model_n_batch,  # Batch size for text generation
            n_threads=8,           # Number of threads to use
            temperature=0.7,       # Sampling temperature
            callbacks=callbacks,   # List of callback handlers
            verbose=False          # Set to True for verbose output
        )
    case "GPT4All_GROOVY":
        # Create an instance of GPT4All
        llm = GPT4All(
            model="/content/drive/MyDrive/Colab/Models/ggml-gpt4all-j-v1.3-groovy.bin", # Path to the GPT4Al Groovy
            max_tokens=model_n_ctx, # Maximum number of tokens in generated text
            backend='gpt2',        # Specify the backend (e.g., 'gptj')
            n_batch=model_n_batch,  # Batch size for text generation
            temperature=0.8,        # Sampling temperature
            callbacks=callbacks,    # List of callback handlers
            verbose=False           # Set to True for verbose output
        )
    case "GPT4All_SNOOZY":
        # Create an instance of GPT4All
        llm = GPT4All(
            model="/content/drive/MyDrive/Colab/Models/GPT4All-13B-snoozy.ggmlv3.q4_0.bin",# Path to the GPT4All model Snoozy
            max_tokens=model_n_ctx, # Maximum number of tokens in generated text
            backend='gptj',        # Specify the backend (e.g., 'gptj')
            n_batch=model_n_batch,  # Batch size for text generation
            temperature=0.75,        # Sampling temperature
            callbacks=callbacks,    # List of callback handlers
            verbose=False           # Set to True for verbose output
        )
    case "VLLM":
        # Create an instance of GPT4All
         llm = VLLM(
          model="mosaicml/mpt-7b",
          trust_remote_code=True,  # mandatory for hf models
          max_new_tokens=128,
          top_k=10,
          top_p=0.95,
          temperature=0.8
        )

    case _default:
        # Raise an exception if the model_type is not supported
        raise Exception(f"Model type {model_type} is not supported. Please choose one of the following: LlamaCpp, GPT4All")

# Explicación de la Configuración de Parámetros

Ruta del Modelo (model_path / model): Especificar la ruta correcta del modelo es crucial para cargar el archivo adecuado desde el sistema de archivos o desde un repositorio remoto.

Tokens Máximos (max_tokens / max_new_tokens): Limitar el número máximo de tokens permite controlar la longitud del texto generado, evitando respuestas excesivamente largas o cortas.

Batch Size (n_batch): Ajustar el tamaño del lote puede mejorar la eficiencia del procesamiento y la capacidad del modelo para manejar múltiples solicitudes simultáneamente.

Backend: Seleccionar el backend adecuado (gptj en este caso) asegura que el modelo se ejecute de manera óptima en la infraestructura disponible.

Callbacks: Los callbacks son útiles para monitorear el progreso y realizar ajustes dinámicos durante la inferencia.

Verbose: Activar o desactivar la salida detallada puede ayudar en la depuración y el monitoreo del comportamiento del modelo.

Parámetros de Generación (top_k, top_p, temperature): Estos parámetros permiten ajustar la aleatoriedad y la diversidad del texto generado. top_k y top_p limitan la selección de palabras, mientras que temperature ajusta la creatividad del modelo.

In [35]:
# Set 'hide_source' to True or False as needed
hide_source = True  # Set to True to hide source documents, False to show them

# Set 'mute_stream' to True or False as needed
mute_stream = True  # Set to True to mute stream output, False to allow it


In [36]:
prompts = [
    "How many Super Bowls have won Tom Brady?",
    "How many Territory sold Tula on 202401?",
    "How many RMAVERICK sold Vehiculos Automotrices De La Piedad on March 2024?",
    "How many ESCAPE NA FHEV sold Zapata (Suc. Pachuca) on 202401?",
    "How many TRUCKS sold Vista Hermosa Laredo Motors on 202402?"
]
expected_answers = [
    "7",
    "15",
    "5",
    "11",
    "20"
]

answers = []
times = []

# Process the prompts
for i, query in enumerate(prompts):
    # Get the answer from the question-answering system
    start = time.time()  # Record the start time for performance measurement

    print(f"\n\n> Question {i+1}:")  # Print the query
    print(query)
    # Create a RetrievalQA instance for question-answering
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever,return_source_documents= False)

    res = qa_chain(query)  # Query the question-answering system using the user's query
    answer, docs = res['result'], [] if hide_source else res['source_documents']  # Extract answer and source documents
    end = time.time()  # Record the end time for performance measurement

    # Store the result
    answers.append(answer)
    times.append(round(end - start, 2))

    # Print the result
    print(f"\n> Expected Answer: {expected_answers[i]}")  # Print the expected answer
    print(f"\n> Model Answer (took {times[-1]} s):")  # Print the model's answer and query response time
    print(answer)

    # Print the relevant sources used for the answer, if not hiding sources
    if not hide_source:
        for document in docs:
            print("\n> " + document.metadata["source"] + ":")  # Print the source document's metadata
            print(document.page_content)  # Print the content of the source document

# Calculate and display the model's performance
average_time = sum(times) / len(times)
print(f"\n\nModel Performance:")
print(f"Average Response Time: {average_time} s")

# Compare model answers with expected answers and calculate accuracy score
correct_answers = 0
for i, answer in enumerate(answers):
    if str(expected_answers[i]).lower() in str(answer).lower():
        correct_answers += 1

accuracy_score = (correct_answers / len(expected_answers)) * 10
print(f"Accuracy Score: {accuracy_score} / 10")




> Question 1:
How many Super Bowls have won Tom Brady?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Expected Answer: 7

> Model Answer (took 55.39 s):
 Tom Brady has won 7 Super Bowls.


> Question 2:
How many Territory sold Tula on 202401?

> Expected Answer: 15

> Model Answer (took 49.08 s):
 15


> Question 3:
How many RMAVERICK sold Vehiculos Automotrices De La Piedad on March 2024?

> Expected Answer: 5

> Model Answer (took 84.54 s):
 The dealer's name is Vehiculos Automotrices De La Piedad. The brand is FORD, and the vehicle line is MAVERICK. The period we are looking for is 202403 (March 2024). However, there isn't any information about the sales of the RMAVERICK in March 2024 provided within these contexts.

Final Answer: I don't know.


> Question 4:
How many ESCAPE NA FHEV sold Zapata (Suc. Pachuca) on 202401?

> Expected Answer: 11

> Model Answer (took 56.93 s):
 11


> Question 5:
How many TRUCKS sold Vista Hermosa Laredo Motors on 202402?

> Expected Answer: 20

> Model Answer (took 99.52 s):
 To find out how many trucks were sold by Vista Hermosa Laredo Motors dur

# Resultado del Modelo
Basándonos en los resultados proporcionados por el modelo LLaMA 2, junto con el contexto adicional de que las respuestas de las preguntas del 2 al 4 provienen de los archivos utilizados para entrenar el modelo, podemos concluir lo siguiente:

El modelo LLaMA 2 demostró ser efectivo para proporcionar respuestas precisas a preguntas simples y directas, como la cantidad de Super Bowls ganados por Tom Brady.
Las respuestas a las preguntas del 2 al 4, que involucran detalles específicos sobre ventas de vehículos en períodos particulares, fueron generadas a partir de la información contenida en los archivos utilizados para entrenar el modelo. Esto sugiere que el modelo ha aprendido y generalizado efectivamente patrones y detalles relevantes de los datos de entrenamiento.
El modelo logró un puntaje de precisión del 80% en la evaluación de las respuestas proporcionadas, lo que indica una capacidad sólida para comprender y responder preguntas con precisión.
Dada la precisión general del modelo y su capacidad para proporcionar respuestas relevantes y detalladas, respaldadas por la información contenida en los archivos de entrenamiento, podemos concluir que el modelo LLaMA 2 es una opción sólida y confiable para este escenario particular de pregunta y respuesta.